In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USER"] = userdata.get('KAGGLE_USER')

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dheblueberrylane","key":"daa4732f33169064632c656f03e9378d"}'}

In [7]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
dataset_path = '../input/the-three-hair-types'

In [10]:
!kaggle datasets download -d vyombhatia/the-three-hair-types

 98% 64.0M/65.3M [00:04<00:00, 23.9MB/s]
100% 65.3M/65.3M [00:04<00:00, 16.3MB/s]


In [11]:
!unzip the-three-hair-types.zip

Archive:  the-three-hair-types.zip
  inflating: Curly Hair/02dac897d1dec9ba8c057a11d041ada8--layered-natural-hair-natural-black-hairstyles.jpg  
  inflating: Curly Hair/08de1b46cf5693bd811b7ca7d72484ff.jpg  
  inflating: Curly Hair/090617-header-mobile.jpg  
  inflating: Curly Hair/0b7aa42d28a71654681bfd13b569ce32.jpg  
  inflating: Curly Hair/0dba94f77ba91fe4c546048f7476c0ed--curly-hair-cuts-bangs-curly-hair-fringe.jpg  
  inflating: Curly Hair/120047603-selfie-photo-of-curly-haired-caucasian-girl-watching-smilingly-into-camera-and-winking-at-it-in-sunn.jpg  
  inflating: Curly Hair/120047608-selfie-photo-of-cute-curly-haired-caucasian-girl-watching-calmly-into-camera-and-flirting-with-it-in.jpg  
  inflating: Curly Hair/12345722_1638604416406814_170629318_n.jpg  
  inflating: Curly Hair/129aa0f078ba4621e7b33bbe68fae650.jpg  
  inflating: Curly Hair/1369382420_42389.jpg  
  inflating: Curly Hair/138370-Frizzy-Curly-Hair.jpg  
  inflating: Curly Hair/18161042_2301279123429535_915369734

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
for dirname, _, filenames in os.walk('../content/hair data'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [22]:
classes = ('straight', 'wavy', 'curly')

transform = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

from PIL import Image

train_datagen = ImageDataGenerator(rescale = 1./255)
image_path = '/content/hair data/Curly Hair/02dac897d1dec9ba8c057a11d041ada8--layered-natural-hair-natural-black-hairstyles.jpg'
image = Image.open(image_path)
img_width, img_height = image.size
train_generator = train_datagen.flow_from_directory('/content/hair data', target_size = (img_height, img_width), batch_size = 32, class_mode = 'binary')



Found 993 images belonging to 3 classes.


In [23]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory('/content/hair data', target_size=(img_height, img_width), batch_size = 32, class_mode = 'binary')

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu',
                                                           input_shape = (img_height, img_width, 3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid')])

model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])



Found 993 images belonging to 3 classes.


In [24]:
history = model.fit(train_generator,
                    steps_per_epoch = train_generator.samples, epochs = 4,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.samples)

test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory('/content/hair data', target_size = (img_height, img_width), batch_size = 4, class_mode = 'binary')

test_loss, test_acc = model.evaluate(test_generator, verbose = 2)

print(f'Test accuracy: {test_acc}')

Epoch 1/4
 32/993 [..............................] - ETA: 2:35 - loss: 0.1288 - accuracy: 0.3273

993/993 [==============================] - 12s 10ms/step - loss: 0.1288 - accuracy: 0.3273 - val_loss: -0.1887 - val_accuracy: 0.3263
Found 993 images belonging to 3 classes.
249/249 - 6s - loss: -1.8874e-01 - accuracy: 0.3263 - 6s/epoch - 24ms/step
Test accuracy: 0.3262839913368225


In [28]:
from tensorflow.keras.preprocessing import image

image_path = '/content/drive/MyDrive/Colab Notebooks/Hackathon/wavy_hair_test_image.jpg'


image_testing = image.load_img(image_path, target_size = (img_height, img_width))

image_array = image.img_to_array(image_testing)
image_array = np.expand_dims(image_array, axis = 0)
image_array = image_array / 255.0

predictions = model.predict(image_array)

predicted_class = np.argmax(predictions)

print(f"Predicted class: {classes[predicted_class]}")


1/1 [==============================] - 0s 20ms/step
Predicted class: straight
